## Image Captioning

#### Mount Google Drive for the caption and images to train on.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import json
import os
import jiwer

In [6]:
# Check if using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


#### Read the caption file from drive and load in dataframe.

In [7]:
# Define the path to your JSON file
json_file_path = '/content/drive/MyDrive/NLP/output_json.json'

# Read the JSON file into a DataFrame
caption_df = pd.read_json(json_file_path, orient='records', lines=True)

# Print the DataFrame to verify
print(caption_df.head())


        image/key                                                IIW  \
0  aar_test_04600  A close-up outdoor shot shows an Echinops Bann...   
1  aar_test_04601  In a low-angle shot, a wall decorated with a p...   
2  aar_test_04602  In an eye-level shot, a brick wall extends fro...   
3  aar_test_04603  A close-up view of a brick wall reveals the wo...   
4  aar_test_04604  A broad, pale-hued shelf encompasses a large m...   

                                       processed_IIW  \
0  a closeup outdoor shot shows an echinops banna...   
1  in a lowangle shot a wall decorated with a pat...   
2  in an eyelevel shot a brick wall extends from ...   
3  a closeup view of a brick wall reveals the wor...   
4  a broad palehued shelf encompasses a large mat...   

                                         summary_IIW  
0  a closeup outdoor shot shows an echinops banna...  
1  a wall decorated with a pattern of square tile...  
2  brick wall extends from the foreground on the ...  
3  a close

#### Read the images from drive and load to the same dataframe

In [8]:
# directory where images are stored
image_dir = '/content/drive/MyDrive/NLP/Image_set/'

# Function to load and preprocess images
def load_and_preprocess_image(image_key):
    image_path = os.path.join(image_dir, f"{image_key}.jpg")
    image = load_img(image_path, target_size=(224, 224))   ## preprocessing, change size to 224x224
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    return image

# Load and preprocess all images
caption_df['image_data'] = caption_df['image/key'].apply(load_and_preprocess_image)


## Extracting features from image using VGG16-ImageNet.

In [9]:
# Load VGG16 model pre-trained on ImageNet
vgg_model = VGG16(weights='imagenet')
vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)

# Function to extract features using VGG16
def extract_features(image_data):
    features = vgg_model.predict(image_data, verbose=0)
    return features

# Extract features for all images
caption_df['image_features'] = caption_df['image_data'].apply(extract_features)

553467096/553467096 [==============================] - 3s 0us/step


## Preparing the Caption data

In [10]:
# Tokenize the captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(caption_df['summary_IIW'])
vocab_size = len(tokenizer.word_index) + 1

# Convert captions to sequences
sequences = tokenizer.texts_to_sequences(caption_df['summary_IIW'])

# Pad sequences
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

## Split Data into Train and Test Sets

In [11]:
from sklearn.model_selection import train_test_split

# Prepare features and captions arrays
features = np.vstack(caption_df['image_features'])
captions = padded_sequences

# Split data into training and testing sets
train_features, test_features, train_captions, test_captions = train_test_split(features, captions, test_size=0.2, random_state=42)

# Image Captioning Model

In [19]:
# Image captioning model
# Image feature extractor
image_input = Input(shape=(4096,))
image_dense = Dropout(0.5)(image_input)
image_dense = Dense(256, activation='relu')(image_dense)

# Sequence processor
caption_input = Input(shape=(max_length,))
caption_embedding = Embedding(vocab_size, 256, mask_zero=True)(caption_input)
caption_lstm = Dropout(0.5)(caption_embedding)
caption_lstm = LSTM(256)(caption_lstm)

# Decoder (feed both outputs into a single model)
decoder = add([image_dense, caption_lstm])
decoder = Dense(256, activation='relu')(decoder)
outputs = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[image_input, caption_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 4096)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 256)             999424    ['input_3[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 4096)                 0         ['input_2[0][0]']             
                                                                                            

# Train the Model

#### Define the parameters

In [17]:
# Define parameters
batch_size = 32
epochs = 20
steps = len(train_features) // batch_size

#### Creating data generators

In [12]:
# Prepare data generators
def data_generator(features, captions, batch_size):
    while True:
        for i in range(0, len(features), batch_size):
            batch_features = features[i:i+batch_size]
            batch_captions = captions[i:i+batch_size]
            X1, X2, y = [], [], []
            for j in range(len(batch_features)):
                seq = batch_captions[j]
                for k in range(1, len(seq)):
                    in_seq, out_seq = seq[:k], seq[k]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(batch_features[j])
                    X2.append(in_seq)
                    y.append(out_seq)
            yield [np.array(X1), np.array(X2)], np.array(y)

In [15]:
# Create train-test data generators
train_generator = data_generator(train_features, train_captions, batch_size)
test_generator = data_generator(test_features, test_captions, batch_size)

#### Model Fit

In [ ]:
 model.fit(train_generator, epochs=epochs, steps_per_epoch=steps, validation_data=test_generator, validation_steps=len(test_features) // batch_size)

Epoch 1/10
10/10 [==============================] - 20s 1s/step - loss: 6.3729 - val_loss: 5.4164
Epoch 2/10
10/10 [==============================] - 7s 737ms/step - loss: 5.2645 - val_loss: 5.3160
Epoch 3/10
10/10 [==============================] - 8s 780ms/step - loss: 5.0707 - val_loss: 5.0397
Epoch 4/10
10/10 [==============================] - 7s 694ms/step - loss: 4.9601 - val_loss: 5.1828
Epoch 5/10
10/10 [==============================] - 8s 801ms/step - loss: 4.8569 - val_loss: 5.0034
Epoch 6/10
10/10 [==============================] - 7s 698ms/step - loss: 4.7772 - val_loss: 5.1245
Epoch 7/10
10/10 [==============================] - 8s 841ms/step - loss: 4.6350 - val_loss: 5.1443
Epoch 8/10
10/10 [==============================] - 7s 699ms/step - loss: 4.5423 - val_loss: 5.0061
Epoch 9/10
10/10 [==============================] - 8s 771ms/step - loss: 4.4562 - val_loss: 5.1643
Epoch 10/10
10/10 [==============================] - 7s 694ms/step - loss: 4.4294 - val_loss: 5.1188


# Save the trained model and tokenizer

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/NLP/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Save the tokenizer to a JSON file
tokenizer_json = tokenizer.to_json()
with open('/content/drive/MyDrive/NLP/tokenizer.json', 'w') as json_file:
    json_file.write(tokenizer_json)

### Introducing Word Error Rate (WER) into the training loop to fine tune the model.
Using WER as a metric to fine-tune an image captioning model helps ensure that the generated captions are as close as possible to the ground-truth captions. By integrating WER into the training loop, you can dynamically adjust the training process to improve model performance, achieve better generalization, and avoid overfitting. This process involves calculating WER for validation data, adjusting learning rates, implementing early stopping, and saving the best model based on WER scores.

In [25]:
def generate_caption(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [26]:
## Implement a function to calculate WER between generated and reference captions.
def calculate_wer(reference, hypothesis):
    return jiwer.wer(reference, hypothesis)

def evaluate_model(model, tokenizer, test_features, test_captions, max_length):
    actual, predicted = list(), list()
    for img_features, caption in zip(test_features, test_captions):
        img_features = np.expand_dims(img_features, axis=0)  # Ensure the correct shape
        yhat = generate_caption(model, tokenizer, img_features, max_length)
        actual_caption = ' '.join([tokenizer.index_word[i] for i in caption if i > 0])
        actual.append(actual_caption)
        predicted.append(yhat)
    # Calculate WER for all predictions
    wer_scores = [calculate_wer(act, pred) for act, pred in zip(actual, predicted)]
    return sum(wer_scores) / len(wer_scores)

# Define the image captioning model, assuming it's already defined and compiled

# Training loop
best_wer = float('inf')
patience = 5
patience_counter = 0


In [28]:
for epoch in range(epochs):
    model.fit(train_generator, epochs=1, steps_per_epoch=steps, validation_data=test_generator, validation_steps=len(test_features) // batch_size)

    # Calculate WER on the validation set
    current_wer = evaluate_model(model, tokenizer, test_features, test_captions, max_length)
    print(f'Epoch {epoch+1}, WER: {current_wer}')

    # Check if the WER has improved
    if current_wer < best_wer:
        best_wer = current_wer
        model.save('best_model.h5')
        patience_counter = 0
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= patience:
        print('Early stopping triggered')
        break

10/10 [==============================] - 8s 747ms/step - loss: 5.1183 - val_loss: 5.2059
Epoch 1, WER: 1.0


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 7s 705ms/step - loss: 5.0684 - val_loss: 5.2184
Epoch 2, WER: 1.0
10/10 [==============================] - 7s 696ms/step - loss: 4.9829 - val_loss: 5.2108
Epoch 3, WER: 1.0
10/10 [==============================] - 8s 767ms/step - loss: 4.8433 - val_loss: 5.1008
Epoch 4, WER: 1.0
10/10 [==============================] - 8s 817ms/step - loss: 4.6194 - val_loss: 5.0409
Epoch 5, WER: 0.987549586803711
10/10 [==============================] - 7s 746ms/step - loss: 4.4137 - val_loss: 5.0239
Epoch 6, WER: 0.9345915196658015
10/10 [==============================] - 7s 714ms/step - loss: 4.3090 - val_loss: 5.3061
Epoch 7, WER: 0.9192453245234994
10/10 [==============================] - 8s 822ms/step - loss: 4.2789 - val_loss: 5.1831
Epoch 8, WER: 0.9259229467284902
10/10 [==============================] - 8s 788ms/step - loss: 4.1745 - val_loss: 5.2237
Epoch 9, WER: 1.0146028403836715
10/10 [==============================] - 8s 812ms/step - loss: 3.8998 

In [29]:
# Save the trained model
model.save('/content/drive/MyDrive/NLP/new_model.h5')